# 🔍 Advanced Search

Fast Search is Relevance AI's most complex search endpoint. 
It combines functionality to search using vectors, exact text search with ability to boost your search results depending on your needs. The following demonstrates a few dummy examples on how to quickly add complexity to your search!

In [1]:
In [1]: %load_ext autoreload
In [2]: %autoreload 2

In [57]:
import pandas as pd
from relevanceai import Client

In [58]:
client = Client()

In [14]:
ds = client.Dataset("clothes")

## Simple Text Search

In [39]:
results = ds.advanced_search(query="nike", fields_to_search=["prod_name"], select_fields=['prod_name'])
pd.DataFrame(results['results'])

,prod_name,_id,_relevance
0,NIKE,44280a84-0e8d-4787-8aa6-d2d9506da894,22.303717
1,NIKE,5e6a78eb-0e1a-4f84-99d1-7362ea4283f9,22.303717
2,NIKE,5a7e502c-618f-42af-9f2b-6658587f8cc2,22.303717
3,NIKE,bdc557f7-1d22-45bc-874d-f09b4fd76928,22.303717
4,NIKE,bb9d74fb-e70f-4cb2-ab35-4925dae8ecac,22.303717
5,NIKE,901b8471-f7af-4dba-a3f3-ecc494f5093a,22.303717
6,NIKE,09b3b546-2b7d-4b44-9b5f-6d7350af2bff,22.303717
7,NIKE,280cb2fb-fc40-4052-acc1-556c08493d24,22.303717
8,NIKE,3c12c230-b2a0-4706-9f4d-7f929ffba714,22.303717
9,NIKE,3b875970-43f3-4342-9372-a9163431c839,22.303717


## Simple Vector Search

In [43]:
# Create a simple mock vector for now
vector = [1e-7] * 512
results = ds.advanced_search(vector_search_query=[
    {"vector":vector,"field":'prod_name_use_vector_'}
], select_fields=['prod_name'])
pd.DataFrame(results['results'])

,prod_name,_id,_relevance
0,PIMA shell trousers,73d95583-29cb-4dae-9066-14cf645195e7,0.130984
1,Rawley Chinos Slim,4e879e37-af82-4a13-80f4-c22b7e9474dc,0.128895
2,Rawley Chinos Slim,46615788-a7af-42ab-9230-ec3087f45217,0.128895
3,Rawley Chinos Slim,8f9ee4ba-81f1-4e95-ba02-3049694308ed,0.128895
4,Rawley Chinos Slim,d4784d29-7aec-46da-9ae7-39431a47a9a1,0.128895
5,Rawley Chinos Slim,d5617367-95e9-40c6-889a-b19b74bb8589,0.128895
6,Rawley Chinos Slim,2362508c-f099-44d1-b14d-0c1490e8eb82,0.128895
7,EDC Eli Kaftan,c21bd3c2-9491-411f-8031-f071da8e0a50,0.128436
8,Ringhild earring pack,c2ec8d6e-6fbd-4601-9a65-9145d784c614,0.128367
9,2PACK SS Body TVP,c6353a97-d8a0-4a5f-8a2d-5deb479a5b25,0.126159


## Combining Text And Vector Search (Hybrid)

Combining text and vector search allows users get the best of both exact text search and contextual vector search. This can be done as shown below.

In [49]:
results = ds.advanced_search(
    query="nike",
    fields_to_search=["prod_name"],
    vector_search_query=[
        {"vector":vector,"field":'prod_name_use_vector_'}
    ],
    select_fields=["prod_name"], # results to return
)
pd.DataFrame(results['results'])

,prod_name,_id,_relevance
0,NIKE,3d13058f-fa09-4f00-bfb0-fecb2671d206,22.365116
1,NIKE,011668c3-5546-458a-a57b-7e270c1dc987,22.365116
2,NIKE,b203ebbb-f75b-45c8-8a45-1d9322f2750d,22.365116
3,NIKE,8bba89a1-b1dd-4a2f-b4e8-68437d7b3c82,22.365116
4,NIKE,890e1643-294e-4fdd-8787-1b0b325c6069,22.365116
5,NIKE,c6def1ca-515d-43c4-8d05-d3de7ebea9b3,22.365116
6,NIKE,a4480651-b9b4-4c02-8a59-c53a9a8f7d13,22.365116
7,NIKE,81c74d7b-0f50-468b-b14e-ba36e9818ca4,22.365116
8,NIKE,7c8f53cf-0c26-416c-891b-095761fb5d38,22.365116
9,NIKE,e9a98454-ced3-4f79-96fd-894684465603,22.365116


## Adjust the weighting of your vector search results

Adjust the weighting of your vector search results to make it easier for you!
Simply add a `weight` parameter your dictionary inside `vector_search_query`.

In [55]:
results = ds.advanced_search(
    query="nike",
    fields_to_search=["prod_name"],
    vector_search_query=[
        {"vector":vector,"field":'prod_name_use_vector_', "weight": 0.5}
    ],
    select_fields=["prod_name"], # results to return
)
pd.DataFrame(results['results'])

,prod_name,_id,_relevance
0,NIKE,3d13058f-fa09-4f00-bfb0-fecb2671d206,22.334417
1,NIKE,011668c3-5546-458a-a57b-7e270c1dc987,22.334417
2,NIKE,b203ebbb-f75b-45c8-8a45-1d9322f2750d,22.334417
3,NIKE,8bba89a1-b1dd-4a2f-b4e8-68437d7b3c82,22.334417
4,NIKE,890e1643-294e-4fdd-8787-1b0b325c6069,22.334417
5,NIKE,c6def1ca-515d-43c4-8d05-d3de7ebea9b3,22.334417
6,NIKE,a4480651-b9b4-4c02-8a59-c53a9a8f7d13,22.334417
7,NIKE,81c74d7b-0f50-468b-b14e-ba36e9818ca4,22.334417
8,NIKE,7c8f53cf-0c26-416c-891b-095761fb5d38,22.334417
9,NIKE,e9a98454-ced3-4f79-96fd-894684465603,22.334417


## Multi-Vector Search Across Multiple Fields

You can easily add more to your search by extending your vector search query as belows.

In [53]:
results = ds.advanced_search(
    query="nike",
    fields_to_search=["prod_name"],
    vector_search_query=[
        {"vector":vector,"field":'prod_name_use_vector_'},
        {"vector":vector,"field":'image_path_clip_vector_'}
    ],
    select_fields=["prod_name"], # results to return
)
pd.DataFrame(results['results'])

,prod_name,_id,_relevance
0,NIKE,890e1643-294e-4fdd-8787-1b0b325c6069,22.390835
1,NIKE,3c12c230-b2a0-4706-9f4d-7f929ffba714,22.390250
2,NIKE,8bba89a1-b1dd-4a2f-b4e8-68437d7b3c82,22.385850
3,NIKE,a4480651-b9b4-4c02-8a59-c53a9a8f7d13,22.385597
4,NIKE,3b875970-43f3-4342-9372-a9163431c839,22.383432
5,NIKE,280cb2fb-fc40-4052-acc1-556c08493d24,22.383057
6,NIKE,81c74d7b-0f50-468b-b14e-ba36e9818ca4,22.377310
7,NIKE,09b3b546-2b7d-4b44-9b5f-6d7350af2bff,22.372906
8,NIKE,901b8471-f7af-4dba-a3f3-ecc494f5093a,22.367360
9,NIKE,b203ebbb-f75b-45c8-8a45-1d9322f2750d,22.365366
